In [50]:
import pandas as pd
import seaborn as sns

yolosac = pd.read_csv('data/yolosac.csv')
yolosac['category'] = yolosac['category'].apply(eval)
print (yolosac['category'])

0                                      [Massage therapist]
1                                      [Massage therapist]
2                                          [Door supplier]
3                                                 [Church]
4                                                 [Church]
                               ...                        
17933                                [American restaurant]
17934                                        [Gas station]
17935                                      [Movie theater]
17936                                   [Pizza restaurant]
17937    [Electronics store, Appliance store, Computer ...
Name: category, Length: 17938, dtype: object


In [51]:
# Top 30 business categories
print(yolosac['category'].explode().value_counts())
unique_cats = yolosac['category'].explode().unique()

category
Restaurant               1352
Fast food restaurant      649
Auto repair shop          513
Park                      463
Mexican restaurant        429
                         ... 
Mechanical contractor       1
Finance broker              1
Stock broker                1
Asphalt contractor          1
Wildlife park               1
Name: count, Length: 2316, dtype: int64


In [54]:
cats_expanded = yolosac['category'].apply(pd.Series)
cats_expanded

,0,1,2,3,4,5,6,7,8,9,10,11
0,Massage therapist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Massage therapist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Door supplier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Church,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Church,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
17933,American restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17934,Gas station,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17935,Movie theater,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17936,Pizza restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Correlation Between Business Categories

In [46]:
def bool_df(item_lists, unique_items):
  bool_dict = {}
  # Iterate over all tags
  for i, item in enumerate(unique_items):
    # Apply Boolean mask
    bool_dict[item] = item_lists.apply(lambda x: item in x)
  
  # Return as dataframe
  return pd.DataFrame(bool_dict)

In [49]:
categories_bool = bool_df(item_lists=yolosac['category'], unique_items=unique_cats)
categories_bool

,Massage therapist,Door supplier,Church,Employment agency,Billiards supply store,Dart supply store,Furniture store,Game store,Sporting goods store,Motor vehicle dealer,...,Animal protection organization,Mfr,Handbags shop,Tapas bar,International airport,Botanical garden,Agriculture,Wine,Restaurant or cafe,Wildlife park
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17933,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17934,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17935,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17936,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [53]:
# Generate correlation table between categories (takes ~2 mins)
categories_corr = categories_bool.corr(method="pearson")
categories_corr

In [63]:
# Testing what categories are similar to 'Massage therapist'
categories_corr['Massage therapist'].sort_values(ascending=False).iloc[:50]

Massage therapist                    1.000000
Massage spa                          0.457389
Reflexologist                        0.387725
Sports massage therapist             0.331955
Aromatherapy service                 0.268527
Day spa                              0.243594
Spa                                  0.229207
Facial spa                           0.225167
Skin care clinic                     0.184547
Foot massage parlor                  0.154999
Thai massage therapist               0.145929
Sauna                                0.134230
Chiropractor                         0.128233
Massage                              0.109595
Pain control clinic                  0.103336
Reiki therapist                      0.103336
Wellness center                      0.096732
Acupuncture clinic                   0.096681
Lymph drainage therapist             0.089071
Holistic medicine practitioner       0.084203
Health consultant                    0.080938
Alternative medicine              